In [3]:
import pandas as pd
import numpy as np

results = pd.read_csv("results.csv")

In [4]:
results

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
47912,2024-11-19,Oman,Iraq,0,1,FIFA World Cup qualification,Muscat,Oman,False
47913,2024-11-19,Kuwait,Jordan,1,1,FIFA World Cup qualification,Kuwait City,Kuwait,False
47914,2024-11-19,Bahrain,Australia,2,2,FIFA World Cup qualification,Riffa,Bahrain,False
47915,2024-11-19,Indonesia,Saudi Arabia,2,0,FIFA World Cup qualification,Jakarta,Indonesia,False


In [17]:
import pandas as pd
import numpy as np

# Load dataset
# Replace 'matches.csv' with the path to your dataset
df = pd.read_csv('results.csv')

# Filter for European countries
european_countries = [
    'Albania', 'Andorra', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium', 
    'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 
    'Denmark', 'Estonia', 'Finland', 'France', 'Faroe Islands', 'Georgia', 'Germany', 'Greece', 'Hungary', 
    'Iceland', 'Ireland', 'Italy', 'Israel', 'Kazakhstan', 'Kosovo', 'Latvia', 'Liechtenstein', 
    'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 
    'North Macedonia', 'Norway', 'Northern Ireland', 'Poland', 'Portugal', 'Romania', 'Russia', 
    'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Scotland', 
    'Turkey', 'Ukraine', 'United Kingdom', 'Vatican City', 'Gibraltar', 'Wales'
]

df = df[df['home_team'].isin(european_countries) & df['away_team'].isin(european_countries)]

# Feature Engineering
# 1. Create a "home_advantage" feature
df['home_advantage'] = np.where(df['neutral'] == False, 1, 0)

# 2. Calculate goal difference as a feature
df['goal_difference'] = df['home_score'] - df['away_score']

# 3. Encode match outcome (target variable)
# 0 = Draw, 1 = Home Win, 2 = Away Win
df['match_outcome'] = df.apply(
    lambda row: 1 if row['home_score'] > row['away_score'] else
                (2 if row['home_score'] < row['away_score'] else 0), axis=1)

# 4. Recent team performance (win rate over last 5 matches)
def calculate_recent_form(team, date, df, window=5):
    recent_matches = df[
        ((df['home_team'] == team) | (df['away_team'] == team)) & (df['date'] < date)
    ].sort_values('date', ascending=False).head(window)
    
    if recent_matches.empty:
        return 0  # No previous matches, assume neutral form
    
    wins = 0
    for _, match in recent_matches.iterrows():
        if (match['home_team'] == team and match['home_score'] > match['away_score']) or \
           (match['away_team'] == team and match['away_score'] > match['home_score']):
            wins += 1
    return wins / window

df['home_team_recent_form'] = df.apply(
    lambda row: calculate_recent_form(row['home_team'], row['date'], df), axis=1)

df['away_team_recent_form'] = df.apply(
    lambda row: calculate_recent_form(row['away_team'], row['date'], df), axis=1)

# 5. Head-to-head statistics
def head_to_head(team1, team2, date, df):
    matches = df[
        ((df['home_team'] == team1) & (df['away_team'] == team2) |
         (df['home_team'] == team2) & (df['away_team'] == team1)) & (df['date'] < date)
    ]
    if matches.empty:
        return 0  # No previous matches, assume neutral
    return matches['goal_difference'].mean()  # Avg goal difference in past matches

df['head_to_head_strength'] = df.apply(
    lambda row: head_to_head(row['home_team'], row['away_team'], row['date'], df), axis=1)

# 6. Tournament encoding
df['tournament_importance'] = df['tournament'].map({
    'FIFA World Cup': 3,
    'UEFA Euro': 3,
    'Friendly': 1,
    'Qualifiers': 2
}).fillna(0)

# Save processed data for modeling
df.to_csv('processed_european_matches.csv', index=False)

print("Feature engineering complete! Processed data saved to 'processed_european_matches.csv'.")


KeyboardInterrupt: 

In [42]:
import pandas as pd

# Assuming 'results' is your DataFrame containing the match data
# Example column names: 'home_score', 'away_score', 'neutral', 'home_team', 'away_team'

# Filter the dataset for non-neutral matches
non_neutral_matches = results[results['neutral'] == False]

# Identify home team wins
home_team_wins = non_neutral_matches[non_neutral_matches['home_score'] > non_neutral_matches['away_score']]
away_team_wins = non_neutral_matches[non_neutral_matches['away_score'] > non_neutral_matches['home_score']]


# Count the number of home team wins
home_team_wins_count = home_team_wins.shape[0]
away_team_wins_count = away_team_wins.shape[0]
draws = european_matches.shape[0] - home_team_wins_count - away_team_wins_count

# Display the result
print(f"Number of home team wins : {home_team_wins_count}")
print(f"Number of away team wins : {away_team_wins_count}")
print(f"Number of draws : {draws}")


Number of home team wins : 4129
Number of away team wins : 2611
Number of draws : 2709


In [78]:
import pandas as pd

# Assuming 'results' is your DataFrame containing the match data
# Example column names: 'home_score', 'away_score', 'tournament'

# Filter the dataset for friendlies
friendlies = results[results['tournament'] == 'Friendly']
print(f'number of friendlies {len(friendlies)}')

# Filter the dataset for non-friendly matches (other tournaments)
other_tournaments = results[results['tournament'] != 'Friendly']
print(f'Other tournaments = {len(other_tournaments)}')

# Sum the total goals scored in friendlies
friendlies_goals = friendlies['home_score'].sum() + friendlies['away_score'].sum()

# Sum the total goals scored in other tournaments
other_tournaments_goals = other_tournaments['home_score'].sum() + other_tournaments['away_score'].sum()

friendlies_count = friendlies.shape[0]  # Number of friendlies
other_tournaments_count = other_tournaments.shape[0]  # Number of other tournaments

# Calculate the average number of goals per match
average_goals_friendlies = friendlies_goals / friendlies_count if friendlies_count > 0 else 0
average_goals_other_tournaments = other_tournaments_goals / other_tournaments_count if other_tournaments_count > 0 else 0

print(average_goals_friendlies)
print(average_goals_other_tournaments)

number of friendlies 3971
Other tournaments = 5478
3.157642911105515
2.940489229645856


In [82]:
uefa_euro_matches = results[results['tournament'] == 'UEFA Euro']

# Sum the total goals scored in UEFA Euro tournaments
uefa_euro_goals = uefa_euro_matches['home_score'].sum() + uefa_euro_matches['away_score'].sum()

# Count the number of UEFA Euro matches
uefa_euro_count = uefa_euro_matches.shape[0]  # Number of UEFA Euro matches

# Calculate the average number of goals per match in UEFA Euro tournaments
average_goals_uefa_euro = uefa_euro_goals / uefa_euro_count if uefa_euro_count > 0 else 0

# Display the results
print(f"Total goals scored in UEFA Euro tournaments: {uefa_euro_goals}")
print(f"Average goals per match in UEFA Euro tournaments: {average_goals_uefa_euro:.2f}")

Total goals scored in UEFA Euro tournaments: 713
Average goals per match in UEFA Euro tournaments: 2.42
